In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import State, ChessAI

# Menschlicher Spieler

Dieses Notebook implementiert einen menschlichen Spieler, welcher für jeden Zug gefragt wird.  

In [ ]:
import chess


class HumanPlayer(ChessAI):
    state: State  # Declare for mypy

    def __init__(self, **kwargs) -> None:
        super().__init__(initial_state=State.STATELESS, **kwargs)

    def make_turn(self, board: chess.Board) -> None:
        """Asks the human to make a turn on the given board."""
        if board.is_fifty_moves() or board.is_game_over():
            return
        next_move = None
        # Create new stats entry
        self.stats.append({"state": State.STATELESS})

        while True:
            next_move_str = input("Enter your next move as UCI string: ")
            try:
                next_move = chess.Move.from_uci(next_move_str)
            except ValueError:
                print("Illegal input!")
                print("Please use UCI format.")
                continue
            if next_move not in board.legal_moves:
                print("This move is not possible!")
                print("Available moves: ", [move.uci() for move in board.legal_moves])
                continue
            break

        # Push next move
        board.push(next_move)
        # Update stats
        self.stats[-1]["move"] = next_move

    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """This method is never called but must be implemented."""
        pass